In [1]:
import requests
import urllib.request
import time
from bs4 import BeautifulSoup

In [103]:
    
root_url = 'https://www.arte.tv/fr/'
r = requests.get(root_url)
soup = BeautifulSoup(r.content, 'html5lib') 
print(soup.findAll('a')) 

[<a class="css-70n7hp eelysnt1" href="">Passer au contenu principal</a>, <a class="css-70n7hp eelysnt1" href="https://www.arte.tv/fr/search/">Passer à la recherche</a>, <a class="css-14zggjp e1yz41mb1" href="https://www.arte.tv/fr/"><span class="sr-only">Accueil</span><svg aria-hidden="true" class="css-e66dxi e35b35k0" focusable="false" tabindex="-1"><use xlink:href="#arte-next-arte"></use></svg></a>, <a class="css-1brmqd2 esxc5lb9" href="https://www.arte.tv/fr/"><div class="css-17piihg esxc5lb8">Accueil</div></a>, <a class="css-1brmqd2 esxc5lb9" href="https://www.arte.tv/fr/guide/"><div class="css-17piihg esxc5lb8">Guide TV</div></a>, <a class="css-1brmqd2 esxc5lb9" href="https://www.arte.tv/fr/direct/"><div class="css-17piihg esxc5lb8">Direct</div></a>, <a class="css-1brmqd2 esxc5lb9" href="https://www.arte.tv/fr/videos/info-et-societe/"><div class="css-17piihg esxc5lb8">Info et société</div></a>, <a class="css-1brmqd2 esxc5lb9" href="https://www.arte.tv/fr/videos/cinema/"><div class

In [126]:
url = 'https://www.arte.tv/fr/videos/info-et-societe/enquetes-et-reportages/'
r = requests.get(url)
soup = BeautifulSoup(r.content, "html5lib")

soup.prettify()

'<!DOCTYPE html>\n<html lang="fr">\n <head prefix="og: http://ogp.me/ns# fb: http://ogp.me/ns/fb#">\n  <title data-helmet="true">\n   Enquêtes et reportages | ARTE\n  </title>\n  <meta charset="utf-8" data-helmet="true"/>\n  <meta content="width=device-width, initial-scale=1.0" data-helmet="true" name="viewport"/>\n  <meta content="ARTE" data-helmet="true" name="copyright"/>\n  <meta content="1040824609280221" data-helmet="true" property="fb:app_id"/>\n  <meta content="fr_fr" data-helmet="true" property="og:locale"/>\n  <meta content="ARTE" data-helmet="true" property="og:site_name"/>\n  <meta content="summary_large_image" data-helmet="true" name="twitter:card"/>\n  <meta content="ARTE" data-helmet="true" name="apple-mobile-web-app-title"/>\n  <meta content="ARTE" data-helmet="true" name="application-name"/>\n  <meta content="#da532c" data-helmet="true" name="msapplication-TileColor"/>\n  <meta content="#ffffff" data-helmet="true" name="theme-color"/>\n  <meta content="Enquêtes et repo

In [125]:
soup.text

'\n        Enquêtes et reportages | ARTE\n        \n        \n            @font-face{\n                font-family: barna;\n                src:\n                    url("//static-cdn.arte.tv/static/fonts/barna-web/woff2/Barna-Regular.woff2") format("woff2"),\n                    url("//static-cdn.arte.tv/static/fonts/barna-web/woff/Barna-Regular.woff") format("woff");\n                font-weight: 400;\n                font-style: normal;\n            }\n            body{\n                font-family: barna, Arial, sans-serif;\n                font-weight: 400;\n                line-height: 1.5;\n                color: #0a0a0a;\n                -webkit-font-smoothing: antialiased;\n                -moz-osx-font-smoothing: grayscal;\n            }\n        \n        \n        window.__CONFIG__ = {"env":"production","app":{"baseUrl":"https:\\u002F\\u002Fwww.arte.tv","version":"20190724105625","assetsBaseUrl":"https:\\u002F\\u002Fstatic-cdn.arte.tv\\u002Fguide","asgCdnBaseUrl":"https:\\u

In [ ]:
from bs4 import BeautifulSoup



In [204]:
import requests
import numpy as np
import pprint, os, sys

def reformat_props(data):

    props = {'title':data['VTI'],
             'duration':float(data['videoDurationSeconds']/60.),
             'language':[],
             'quality':[],
             'url':[]}
    if 'VDE' in data:
        props['summary'] = data['VDE']
    if 'V7T' in data:
        props['synopsis'] = data['V7T']
             
    for key in data['VSR']:
        props['language'].append(data['VSR'][key]['versionShortLibelle'])
        props['quality'].append(str(data['VSR'][key]['width'])+'x'+\
                                                    str(data['VSR'][key]['height']))    
        props['url'].append(data['VSR'][key]['url'])
    
    props['language'] = np.array(props['language'])
    props['quality'] = np.array(props['quality'])
    props['url'] = np.array(props['url'])
    
    if props['title'] == 'ARTE Reportage':
        props['title'] += ' '+data['subtitle']

    props['reformated_title'] = props['title'].replace('  ', '_').replace(' ','_').replace('/', '|')

    return props

    def reformat_title(title):
        return 

    
class video:
    
    global api_url
    api_url = 'https://api.arte.tv/api/player/v1/config/fr/'

        
    def __init__(self, ID):
        self.ID = ID
        
        req = requests.get(api_url+self.ID)
        if True:
            self.data = req.json()['videoJsonPlayer']
            self.props = reformat_props(self.data)
        else:
            self.props = {}
                    
    def show_props(self):
        pprint.pprint(self.props)
        
    def pick_quality_and_languages(self):
        self.quality = quality
        self.languages = languages
    
    def download(self,
                 quality = '640x360',
                 languages = ['VOF', 'VOSTF'],
                 folder='/media/yann/DATA/Arte/',
                 chunk_size = 1024*1024):
        
        file_url = ''
        for l in languages:
            cond = (self.props['language']==l) & (self.props['quality']==quality)
            if np.sum(cond)>0:
                file_url = self.props['url'][cond][0]
                break
            
        # create response object 
        r = requests.get(file_url, stream = True) 
          
        total_length = r.headers.get('content-length')

        # download started 
        with open(os.path.join(folder,self.props['reformated_title']+'.mp4'), 'wb') as f: 
                    
            print('%s (%.0f min)' % (self.props['title'],self.props['duration']))
            if total_length is None: # no content length header
                f.write(r.content)
            else:
                dl = 0
                total_length = int(total_length)
                for data in r.iter_content(chunk_size=chunk_size):
                    dl += len(data)
                    f.write(data)
                    done = int(50 * dl / total_length)
                    sys.stdout.write("\r[%s%s]" % ('=' * done, ' ' * (50-done)) )    
                    sys.stdout.flush()                
                    


Une espèce à part - Vie unique (3 min)
[==================================================]

In [203]:
f = open('/home/yann/Desktop/arte.txt')
IDS = []
l = f.readline()
while len(l)>10:
    IDS.append(l.split('https://www.arte.tv/fr/videos/')[1][:12])
    l = f.readline()
    
for link in IDS:
    vid = video(link)
    print('%s (%.0f min)' % (vid.props['title'], vid.props['duration']))


ARTE Reportage Tanzanie / Israël (52 min)
ARTE Reportage Tchad / Inde (52 min)
Océans, le grand kidnapping (89 min)
ARTE Regards - Que faire des anciens djihadistes européens ? (30 min)
ARTE Reportage Mexique : à la recherche des migrants disparus (52 min)
ONU, le scandale des abus sexuels (52 min)
ARTE Regards - Un championnat automobile féminin (30 min)
ARTE Regards - Revenir du djihad (30 min)
Les nouveaux mercenaires russes (87 min)
La meilleure du monde (95 min)
Copie conforme (102 min)
Rendez-vous (80 min)
Théorème (92 min)
Main basse sur la ville (96 min)
L'avventura (133 min)
La Paloma (107 min)
Marie-Thérèse d'Autriche - Impératrice et mère (50 min)
Maximilien d'Autriche - Amour et pouvoir à la Renaissance (51 min)
The Last Movie (104 min)
George Michael : Freedom (91 min)
Les amours imaginaires (97 min)
Le rayon vert (95 min)
La fiancée du pirate (102 min)
Révolutions sexuelles (1/2) (53 min)
Révolutions sexuelles (2/2) (54 min)
Tu enfanteras dans la douleur (59 min)
Question

In [174]:
url = 'https://www.arte.tv/fr/videos/info-et-societe/enquetes-et-reportages/'
#url = 'https://www.arte.tv/fr/videos/plus-vues/'

s = requests.Session()
r = s.post(url)
for i in range(3):
    r = s.get(url+'?page='+str(i+1))
    soup = BeautifulSoup(r.content, "html5lib")
    L = soup.prettify().split('href=\"/fr/videos/')

    Link_List = [l[:12] for l in L[1:]]
    for l in Link_List:
        if (l[10:12]=='-A'):
            vid = video(l)
            print('\n====================================================')
            print(' ', vid.props['title'])
            #print(' ------------------------------------------')
            #print(vid.props['synopsis'])
        else:
            print(l)


  Tanzanie : l'enfance volée des albinos

  Israël : cannabis, la terre promise

  Inde : les ponts vivants du Meghalaya

  Océans, le grand kidnapping

  Sénégal : pilleurs des mers

  Cap-Vert : les forçats du sable

  ARTE Reportage

  ARTE Reportage

  Inde : les ponts vivants du Meghalaya

  ARTE Reportage

  Russie : guerre et pont

  ARTE Reportage

  Brésil : un corridor nommé BR-163

  ARTE Regards - Rondes et belles

  ARTE Regards - La maison de retraite des musiciens

  ARTE Regards - Un barrage menace la nature en Albanie

  Tanzanie : l'enfance volée des albinos

  Israël : cannabis, la terre promise

  Inde : les ponts vivants du Meghalaya

  Océans, le grand kidnapping

  Sénégal : pilleurs des mers

  Cap-Vert : les forçats du sable

  ARTE Reportage

  ARTE Reportage

  Inde : les ponts vivants du Meghalaya

  ARTE Reportage

  Russie : guerre et pont

  ARTE Reportage

  Brésil : un corridor nommé BR-163

  ARTE Regards - Rondes et belles

  ARTE Regards - La maison